In [3]:
import numpy as np
import pandas as pd

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode()


In [8]:
! wget https://raw.githubusercontent.com/livcms/hlt_timing/master/sen_constPU0_numEvent100_timings.csv
! wget https://raw.githubusercontent.com/livcms/hlt_timing/master/sen_constPU140_numEvent100_timings.csv
! wget https://raw.githubusercontent.com/livcms/hlt_timing/master/sen_constPU200_numEvent100_timings.csv

--2020-04-21 08:47:50--  https://raw.githubusercontent.com/livcms/hlt_timing/master/sen_constPU0_numEvent100_timings.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.236.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.236.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6029 (5.9K) [text/plain]
Saving to: ‘sen_constPU0_numEvent100_timings.csv.3’

sen_constPU0_numEve 100%[===================>]   5.89K  --.-KB/s    in 0.001s  

2020-04-21 08:47:50 (8.86 MB/s) - ‘sen_constPU0_numEvent100_timings.csv.3’ saved [6029/6029]

--2020-04-21 08:47:50--  https://raw.githubusercontent.com/livcms/hlt_timing/master/sen_constPU140_numEvent100_timings.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.236.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.236.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6169 (6.0K) [text/pla

In [9]:
pu0=pd.read_csv("sen_constPU0_numEvent100_timings.csv")
pu140 =pd.read_csv("sen_constPU140_numEvent100_timings.csv")
pu200 = pd.read_csv("sen_constPU200_numEvent100_timings.csv")

In [10]:
def categorise_modules(module):
    if "cutsReco" in module: 
        return "cutsReco"
    elif "trackValidator" in module: 
        return "trackValidator"
    elif "siPixel" in module: 
        return "siPixel"
    elif "initialStep" in module: 
        return "initialStep"
    elif "highPt" in module: 
        return "highPt"
    elif "ecal" in module: 
        return "ecal"
    else: 
        return "other"

In [11]:

#pu0.drop("Std (ms) at 100 events", axis=1, inplace=True)
allpu = pu0.copy()
allpu["PU140"] = pu140["Average time/event (ms) at 100 events"]
allpu["PU200"] = pu200["Average time/event (ms) at 100 events"]
allpu["ModuleType"] = allpu["Module"].apply(lambda x: categorise_modules(x))
allpu.rename(columns={'Average time/event (ms) at 100 events':'PU0'}, inplace=True)

In [12]:
def nlargest_three(x): 
    largestpu0 = allpu.nlargest(x, "PU0")["Module"]
    largestpu140 = allpu.nlargest(x, "PU140")["Module"]
    largestpu200 = allpu.nlargest(x, "PU200")["Module"]
    
    nlongest_mod = np.unique((largestpu0.append(largestpu140)).append(largestpu200))
    #df = dt.loc[allpu['Module'].isin(nlongest_mod)]
    return nlongest_mod


In [65]:
def plot1():
    num_steps = len(np.arange(0,151,10))
    trace_list1 = [
        # Create subplots: use 'domain' type for Pie subplot
        go.Pie(values=allpu.nlargest(step, "PU0")["PU0"][5:],
               labels=allpu.nlargest(step, "PU0")["Module"][5:],
               domain={'x':[0.1,0.9], 'y':[0.05,0.9]},
               hole=0.5,
               direction='clockwise',
               sort=True,
               marker=dict(line=dict(color='#000000', width=0.2))
                  ) for step in np.arange(0,151,10)]

    trace_list2 = [
        # Create subplots: use 'domain' type for Pie subplot
        go.Pie(values=allpu.nlargest(step, "PU140")["PU0"][5:],
               labels=allpu.nlargest(step, "PU140")["Module"][5:],
               domain={'x':[0.1,0.9], 'y':[0.05,0.9]},
               hole=0.75,
               direction='clockwise',
               sort=True,
               marker=dict(line=dict(color='#000000', width=0.2))
                  ) for step in np.arange(0, 151, 10)]

    trace_list3 = [
        # Create subplots: use 'domain' type for Pie subplot
        go.Pie(values=allpu.nlargest(step, "PU200")["PU0"][5:],
               labels=allpu.nlargest(step, "PU200")["Module"][5:],
               domain={'x':[0.1,0.9], 'y':[0.05,0.9]},
               hole=0.9,
               direction='clockwise',
               sort=True,
               marker=dict(line=dict(color='#000000', width=0.2))
                  ) for step in np.arange(0, 151, 10)]


    fig = go.Figure(data=trace_list1+trace_list2+trace_list3)
    fig.update_traces(textposition= "none", textinfo=None)


    steps = []
    for i in range(num_steps):
        # Hide all traces
        step = dict(
            method = 'restyle',  
            args = ['visible', [False] * len(fig.data)],
        )
        # Enable the two traces we want to see
        step['args'][1][i] = True
        step['args'][1][i+num_steps] = True
        step['args'][1][i+2*num_steps] = True

        # Add step to step list
        steps.append(step)

    sliders = [dict(
        steps = steps,
        active=10,
        currentvalue={"prefix": "Showing the "},
        pad={"t": 50},
    )]


    fig.layout.sliders = sliders

    fig['layout']['sliders'][0]['currentvalue']['suffix']=' longest modules in ms: '
    for i, date in enumerate(np.arange(0,151,10)):
        fig['layout']['sliders'][0]['steps'][i]['label']=str(date)

    fig.update_layout(
        title="n longest modules at PU 0,140,200")
    
    iplot(fig, show_link=False)

In [26]:
#import chart_studio as py
def plot2(nmods):

    nmods =20 

    trace_list1 = go.Pie(values=allpu.nlargest(nmods, "PU0")["PU0"][5:],
           labels=allpu.nlargest(nmods, "PU0")["Module"][5:],
           domain={'x':[0.1,0.9], 'y':[0.05,0.9]},
           hole=0.5,
           direction='clockwise',
           sort=True,
           marker=dict(line=dict(color='#000000', width=0.2))
              )

    trace_list2 = go.Pie(values=allpu.nlargest(nmods, "PU140")["PU0"][5:],
           labels=allpu.nlargest(nmods, "PU140")["Module"][5:],
           domain={'x':[0.1,0.9], 'y':[0.05,0.9]},
           hole=0.75,
           direction='clockwise',
           sort=True,
           marker=dict(line=dict(color='#000000', width=0.2))
              )

    trace_list3 =  go.Pie(values=allpu.nlargest(nmods, "PU200")["PU0"][5:],
           labels=allpu.nlargest(nmods, "PU200")["Module"][5:],
           domain={'x':[0.1,0.9], 'y':[0.05,0.9]},
           hole=0.9,
           direction='clockwise',
           sort=True,
           marker=dict(line=dict(color='#000000', width=0.2))
              )

    data = [trace_list1, trace_list2, trace_list3]



    updatemenus = list([
    dict(type="buttons",
         active=-1,
         buttons=list([
            dict(label = 'PU0',
                 method = 'update',
                 args = [{'visible': [True, False, False]}]),

            dict(label = 'PU140',
                 method = 'update',
                 args = [{'visible': [False, True, False]}]),

            dict(label = 'PU200',
                 method = 'update',
                 args = [{'visible': [False, False, True]}]),

            dict(label = 'PU0 and PU140',
                 method = 'update',
                 args = [{'visible': [True, True, False]}]),

            dict(label = 'PU0 and 200',
                 method = 'update',
                 args = [{'visible': [True, False, True]}]),

            dict(label = 'PU140 and 200',
                 method = 'update',
                 args = [{'visible': [False, True, True]}]),

            dict(label = 'All',
                 method = 'update',
                 args = [{'visible': [True, True, True]}])
        ]),
    )
    ])





    layout = dict(title='Compare n modules for selected pileups', showlegend=False,
              updatemenus=updatemenus)

    fig2 = dict(data=data, layout=layout)
    
    iplot(fig2, filename='update_button')

In [30]:
itLocalReco =[
    'siPhase2Clusters', 
    'siPixelClusters', 
    'siPixelClusterShapeCache',
    'siPixelClustersPreSplitting', 
    'siPixelRecHits', 
    'siPixelRecHitsPreSplitting'
]

In [31]:
caloLocalReco = [
    'hbhereco',
    'hfprereco',
    'hfreco',
    'horeco',
    'ecalUncalibRecHitSequence',
    'ecalRecHit'
]

In [32]:
initialStepSequence = [
    'initialStepSeedLayers', 
    'initialStepTrackingRegions', 
    'initialStepHitDoublets',
    'initialStepHitQuadruplets', 
    'initialStepSeeds', 
    'initialStepTrackCandidates', 
    'initialStepTracks',
    'initialStepPVSequence', #uses beamspot
    'initialStepSelector']

In [33]:
highPtTripletStepSequence = [
    'highPtTripletStepClusters',
    'highPtTripletStepSeedLayers',
    'highPtTripletStepTrackingRegions',
    'highPtTripletStepHitDoublets',
    'highPtTripletStepHitTriplets',
    'highPtTripletStepSeedLayers',
    'highPtTripletStepSeeds',
    'highPtTripletStepTrackCandidates',
    'highPtTripletStepTracks',
    'highPtTripletStepSelector',
    'initialStepSeedClusterMask',# needed by electron, but also by highPtTripletStepSeedClusterMask
    'highPtTripletStepSeedClusterMask']

In [34]:
vertexReco = [
    'ak4CaloJetsForTrk',
    'unsortedOfflinePrimaryVertices',
    'trackWithVertexRefSelectorBeforeSorting',
    'trackRefsForJetsBeforeSorting',
    'offlinePrimaryVertices',
    'offlinePrimaryVerticesWithBS',
    'inclusiveVertexFinder',
    'vertexMerger',
    'trackVertexArbitrator',
    'inclusiveSecondaryVertices'
]

In [35]:
DQM_offline = ['PV0p1',
               'TrackMon_ckf',
               'TrackSeedMonhighPtTripletStep',
               'TrackSeedMoninitialStep', 
               'TrackSeedMonlowPtQuadStep', 
               'TrackSplitMonitor', 
               'TrackerCollisionSelectedTrackMonCommongeneralTracks',
               'TrackerCollisionSelectedTrackMonCommonhighPurityPV0p1', 
               'TrackerCollisionSelectedTrackMonCommonhighPurityPt1', 
               'TrackerCollisionSelectedTrackMonCommonhighPurityPtRange0to1',
               'dqmInfoTracking', 
               'highPurityPV0p1', 
               'highPurityPt1',
               'highPurityPtRange0to1', 
               'primaryVertexResolution', 
               'pvMonitor', 
               'trackingDQMgoodOfflinePrimaryVertices'
]

In [36]:
preval = [
'trackValidatorTrackingOnly',
'trackValidatorTPPtLess09Standalone',
'trackValidatorFromPVStandalone',
'trackValidatorFromPVAllTPStandalone',
'trackValidatorAllTPEfficStandalone', 
'trackValidatorBHadronTrackingOnly', 
'trackValidatorSeedingTrackingOnly', 
'trackValidatorSeedingPreSplittingTrackingOnly', 
'trackValidatorBuilding', 
'trackValidatorBuildingPreSplitting', 
'VertexAssociatorByPositionAndTracks', 
'cutsRecoTracksBtvLike', 
'cutsRecoTracksFromPVHighPtTripletStep', 
'cutsRecoTracksFromPVHighPtTripletStepHp', 
'cutsRecoTracksFromPVHp',
'cutsRecoTracksFromPVInitialStep', 
'cutsRecoTracksFromPVInitialStepHp', 
'cutsRecoTracksFromPVLowPtQuadStep', 
'cutsRecoTracksFromPVLowPtQuadStepHp', 
'cutsRecoTracksFromPVPt09HighPtTripletStep', 
'cutsRecoTracksFromPVPt09HighPtTripletStepHp', 
'cutsRecoTracksFromPVPt09Hp', 
'cutsRecoTracksFromPVPt09InitialStep', 
'cutsRecoTracksFromPVPt09InitialStepHp', 
'cutsRecoTracksFromPVPt09LowPtQuadStep', 
'cutsRecoTracksFromPVPt09LowPtQuadStepHp',  
'cutsRecoTracksHighPtTripletStep', 
'cutsRecoTracksHighPtTripletStepByAlgoMask', 
'cutsRecoTracksHighPtTripletStepByAlgoMaskHp', 
'cutsRecoTracksHighPtTripletStepByOriginalAlgo', 
'cutsRecoTracksHighPtTripletStepByOriginalAlgoHp', 
'cutsRecoTracksHighPtTripletStepHp',
'cutsRecoTracksHp',
'cutsRecoTracksInitialStep',
'cutsRecoTracksInitialStepByAlgoMask',
'cutsRecoTracksInitialStepByAlgoMaskHp',
'cutsRecoTracksInitialStepByOriginalAlgo',
'cutsRecoTracksInitialStepByOriginalAlgoHp',
'cutsRecoTracksInitialStepHp',
'cutsRecoTracksLowPtQuadStep',
'cutsRecoTracksLowPtQuadStepByAlgoMask',
'cutsRecoTracksLowPtQuadStepByAlgoMaskHp',
'cutsRecoTracksLowPtQuadStepByOriginalAlgo',
'cutsRecoTracksLowPtQuadStepByOriginalAlgoHp',
'cutsRecoTracksLowPtQuadStepHp',
'cutsRecoTracksPt09HighPtTripletStep',
'cutsRecoTracksPt09HighPtTripletStepHp',
'cutsRecoTracksPt09H',
'cutsRecoTracksPt09InitialStep',
'cutsRecoTracksPt09InitialStepHp',
'cutsRecoTracksPt09LowPtQuadStep', 
'cutsRecoTracksPt09LowPtQuadStepHp',
'generalTracksFromPV',
'generalTracksFromPVPt09',
'generalTracksPt09',
'quickTrackAssociatorByHits',
'quickTrackAssociatorByHitsPreSplitting',
'seedTrackshighPtTripletStepSeeds',
'seedTracksinitialStepSeeds',
'seedTrackslowPtQuadStepSeeds', 
'tpClusterProducer',
'tpClusterProducerPreSplitting',
'trackingParticleNumberOfLayersProducer',
'trackingParticleRecoTrackAsssociation',
'trackingParticlesBHadron', 
'trackingParticlesConversion', 
'trackingParticlesElectron', 
'trackingParticlesSignal',
'selectedOfflinePrimaryVertices',
'selectedOfflinePrimaryVerticesWithBS', 
'simHitTPAssocProducer', 
'v0Validator', 
'vertexAnalysisTrackingOnly'
'cutsRecoTracksPt09Hp', 
'vertexAnalysisTrackingOnly'
]

In [37]:
raw_2digi= [
    'ecalDigis', 
    'hcalDigis', 
    'muonCSCDigis', 
    'muonDTDigis', 
    'muonGEMDigis', 
    'muonRPCDigis', 
    'muonRPCNewDigis', 
    'scalersRawToDigi'
]

In [38]:
def categorise_modules2(module):
    if module in itLocalReco: 
        return "itLocalReco"
    elif module in caloLocalReco: 
        return "caloLocalReco"
    elif module in initialStepSequence: 
        return "initialStepSequence"
    elif module in highPtTripletStepSequence: 
        return "highPtTripletStepSequence"
    elif module in vertexReco: 
        return "vertexReco"
    elif module in DQM_offline:
        return "DQM_offline"
    elif module in preval: 
        return "preval"
    elif module in raw_2digi: 
        return "raw_2digi"
    else: 
        return "other"

In [39]:
allpu_mt = allpu.copy()

allpu_mt["ModuleType"] = allpu_mt["Module"].apply(lambda x: categorise_modules2(x.strip()))

In [90]:
#pd.set_option('display.max_rows', df.shape[0]+1)
#tmp[tmp["ModuleType"]=="other"]

In [91]:
def plot3():

    sumbyModuleType = allpu_mt[5:].groupby(['ModuleType']).sum().reset_index()

    trace1 = [go.Pie(values=sumbyModuleType["PU0"],
               labels=sumbyModuleType["ModuleType"],
               domain={'x':[0.1,0.9], 'y':[0.05,0.9]},
               hole=0.5,
               direction='clockwise',
               sort=True,
               marker=dict(line=dict(color='#000000', width=0.2))
                  )]


    trace2 =   [go.Pie(values=sumbyModuleType["PU140"],
               labels=sumbyModuleType["ModuleType"],
               domain={'x':[0.1,0.9], 'y':[0.05,0.9]},
               hole=0.75,
               direction='clockwise',
               sort=True,
               marker=dict(line=dict(color='#000000', width=0.2))
                  )]

    trace3 =  [go.Pie(values=sumbyModuleType["PU200"],
               labels=sumbyModuleType["ModuleType"],
               domain={'x':[0.1,0.9], 'y':[0.05,0.9]},
               hole=0.9,
               direction='clockwise',
               sort=True,
               marker=dict(line=dict(color='#000000', width=0.2))
                  )]

    fig3 = go.Figure(data=trace1+trace2+trace3)


    fig3.update_layout(
        title="Total for module types at PU0, 140, 200")


    fig3.update_traces(textposition= "none", textinfo=None)


    iplot(fig3)

In [95]:
import warnings
warnings.filterwarnings("ignore")

def plot4(pu):
    step =20 

    col = "PU"+str(pu)

    trace_list1 = go.Pie(values=allpu_mt[allpu_mt["ModuleType"]=="DQM_offline"][col],
            labels=allpu_mt[allpu_mt["ModuleType"]=="DQM_offline"]["Module"],
               domain={'x':[0.1,0.9], 'y':[0.05,0.9]},
               hole=0.5,
               direction='clockwise',
               sort=True,
               marker=dict(line=dict(color='#000000', width=0.2))
                  )

    trace_list2 = go.Pie(values=allpu_mt[allpu_mt["ModuleType"]=="preval"][col],
            labels=allpu_mt[allpu_mt["ModuleType"]=="preval"]["Module"],
               domain={'x':[0.1,0.9], 'y':[0.05,0.9]},
               hole=0.5,
               direction='clockwise',
               sort=True,
               marker=dict(line=dict(color='#000000', width=0.2))
                  )

    trace_list3 = go.Pie(values=allpu_mt[allpu_mt["ModuleType"]=="raw_2digi"][col],
            labels=allpu_mt[allpu_mt["ModuleType"]=="raw_2digi"]["Module"],
               domain={'x':[0.1,0.9], 'y':[0.05,0.9]},
               hole=0.5,
               direction='clockwise',
               sort=True,
               marker=dict(line=dict(color='#000000', width=0.2))
                  )

    trace_list4 = go.Pie(values=allpu_mt[allpu_mt["ModuleType"]=="caloLocalReco"][col],
            labels=allpu_mt[allpu_mt["ModuleType"]=="caloLocalReco"]["Module"],
               domain={'x':[0.1,0.9], 'y':[0.05,0.9]},
               hole=0.5,
               direction='clockwise',
               sort=True,
               marker=dict(line=dict(color='#000000', width=0.2))
                  )

    trace_list5 = go.Pie(values=allpu_mt[allpu_mt["ModuleType"]=="initialStepSequence"][col],
            labels=allpu_mt[allpu_mt["ModuleType"]=="initialStepSequence"]["Module"],
               domain={'x':[0.1,0.9], 'y':[0.05,0.9]},
               hole=0.5,
               direction='clockwise',
               sort=True,
               marker=dict(line=dict(color='#000000', width=0.2))
                  )

    trace_list6 = go.Pie(values=allpu_mt[allpu_mt["ModuleType"]=="highPtTripletStepSequence"][col],
            labels=allpu.nlargest(step, "PU0")[allpu_mt["ModuleType"]=="highPtTripletStepSequence"]["Module"],
               domain={'x':[0.1,0.9], 'y':[0.05,0.9]},
               hole=0.5,
               direction='clockwise',
               sort=True,
               marker=dict(line=dict(color='#000000', width=0.2))
                  )

    trace_list7 = go.Pie(values=allpu_mt[5:].nlargest(step, "PU0")[allpu_mt[5:]["ModuleType"]=="other"][col],
            labels=allpu_mt[5:].nlargest(step, "PU0")[allpu_mt[5:]["ModuleType"]=="other"]["Module"],
               domain={'x':[0.1,0.9], 'y':[0.05,0.9]},
               hole=0.5,
               direction='clockwise',
               sort=True,
               marker=dict(line=dict(color='#000000', width=0.2))
                  )
    data = [trace_list1, trace_list2, trace_list3, trace_list4, trace_list5, trace_list6, trace_list7]



    updatemenus = list([
        dict(type="buttons",
             active=-1,
             buttons=list([
                dict(label = 'DQM_offline',
                     method = 'update',
                     args = [{'visible': [True, False, False, False, False, False, False]}]),

                dict(label = 'preval',
                     method = 'update',
                     args = [{'visible': [False, True, False, False, False, False, False]}]),

                dict(label = 'raw_digi',
                     method = 'update',
                     args = [{'visible': [False, False, True, False, False, False, False]}]),

                dict(label = 'caloLocalReco',
                     method = 'update',
                     args = [{'visible': [False, False, False, True, False, False, False]}]),

                dict(label = 'initialStepSequence',
                     method = 'update',
                     args = [{'visible': [False, False, False, False, True, False, False]}]),

                dict(label = 'highPtTripletStepSequence',
                     method = 'update',
                     args = [{'visible': [False, False, False, False, False, True, False]}]),

                 dict(label = 'other',
                     method = 'update',
                     args = [{'visible': [False, False, False, False, False, False, True]}]),

                dict(label = 'All',
                     method = 'update',
                     args = [{'visible': [True, True, True, True, True, True, True]}])
            ]),
        )
    ])





    layout = dict(title='Modules by path for '+col, showlegend=False,
                  updatemenus=updatemenus)

    fig = dict(data=data, layout=layout)
    iplot(fig, filename='update_button')

# RESULTS

## Fig 1 - n longest modules at various PUs 

Inner circle is no pileup, middle is PU140, outer is PU200. 

In [66]:
plot1()

## Fig. 2 - Compare n modules at selected pileups 

In [27]:
#the argument to this function is the number of modules you want to see in each PU. Selects the n longest
plot2(10)


## Fig. 3 - Sum of modules by module path 

Each module belongs to a specific path. The paths _initialStepSequence_, _highPtTripletStepSequence_ and most of the modules in _other_ are part of the tracking. 

In [92]:
plot3()

# Fig. 4 - Select modules by path for selected pileup 

In [96]:
#number in the bracket is pileup
plot4(200)